### The following is from [HERE](https://geographicdata.science/book/notebooks/11_regression.html), then modified as the example is worked through

## Spatial Regression
Regression (and predicion more generally), provides us a perfect case to examine how spatial structure can help us inderstand and analyze our data. In this chaprer we discuss how spatial structure can be used to both validate and imprive predicion algorithms, ficusing on linear regression speciffically. 

### What is spatial regression and why should I care?

Usually, Spatial structure helps regression models in one of two ways, The first (and most clear) way space can have an impact on our data is when the process *generating* the data is itself inherently spatial. Here, thisk of somthing like the prices ofr a single family home. It;s often the case that individuals pay a premiium on their house price in order to live in a better school district for the same quality house. Alternativly, homes closer to noie or chemical polluters like waste watter tretment plants, recycling facilities, or wide highways, may actuallu be cheaper than we would otherwise anticipate. In cases like aethma incidence, the locations individuals tend to travel to throughout the day - such as their work of areas of recreation- mau have more of an impact on their health than their given residential address. In this case, it may be necessary to use the data *from other sites* to predict the asthema incidence at a given site. Regardless of the specific case at play here, *geography is a feature*: it directly helps us make predictions about outcomes *because the outcomes are obtained from a geographical process*

An alternative (and more skeptical understanding) reluctantly acknoledged geography's instrumental value. Often, in the analysis of predictive methods and classifiers, we are interested in analyzing what we get wrong. This in common in econometrics; an analyst may be concerned that the model *systematically* mis-predicts somme types of observations. If we know our model routinely performs poorly on a known set of observations or type fo input, we might make a better model if we can account for this. Among other kinds of error diagnostics, geography provides us with an explicitly useful embedding to assess structure in our errors. Mapping classification/prediction error can help us show whether or not there are *clusters of error* in our data. If we *know* that errors tend to be larger in some areas than other areas (or if error is "contigious" between observations), then we might be able to exploit this structure to bake better predictions. 

Spatial structure in our errors might arise from when geography *should be* an attribute somehow, but we are not sure exactally how to include it in our model. THey may also arise because there is some *other* feature whose omission causes the spatial patterns in the error we see. If this additional feature were included, the structure would disapper. Or, it might arise from the complex interactions and interdependencies between the features that we have chosen as predictors., resulting in an intrinsic structure in mis-prediction. Most of the predictors we use in models of social processes contain *embodied* spatial information: patterning ontrinsic to the feature that er get for free in the model. If we intend to or not, using a spatailly patterened predictor in a model can result in spatially patterned errorsl using more thatn one can amplify this effect. Thus, *regardless of whether or not the true preocess is explicitly geographic*, additional information about the spatial relationships between our observations or more information about nearby sites can make our predictions better. 

In this notebook, we build space into the traditional regression framework. We begin with a standard linear regression model, devoid of andy geographical reference. From there, we formalize space and spatial relationships in three main ways: 
* encoding it in exogenous variables
* through spatial heterogeneity, or as systematic variation of outcomes across space
* as dependence, or through the effect associated th the characteristics of spatial neighbors.
  
Throughout, we focus on the conceptual difffernces each approach entails rather than on the technical details. 

In [1]:
from pysal.lib import weights
from pysal.explore import esda
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/home/skye/miniconda3/envs/Spatial_stats/lib/python3.11/site-packages/spaghetti/network.py:42: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


### Data: San Diego Airbnb

To learn a little about how regressoin works, we'll examine information about Airbnb properties in San Diego, CA. THis dataset contains house intrinsic charateristics, both continious (number of beds as in beds) and categorical (type of renting, of, in Airbnb jargon, property group as in the series of pg_X binary variables), but also variables that explicitly refer to the location and spatial configuration of the dataset (e.g., distance to Balboa Park, d2balboa or neighborhood id, neighborhood_cleansed)

In [2]:
db = gpd.read_file("../data/air_bnb/regression_db.geojson")

These are the explanatory variables we will use throughout the notebook

In [3]:
variable_names = [
    "accommodates",  # Number of people it accommodates
    "bathrooms",  # Number of bathrooms
    "bedrooms",  # Number of bedrooms
    "beds",  # Number of beds
    # Below are binary variables, 1 True, 0 False
    "rt_Private_room",  # Room type: private room
    "rt_Shared_room",  # Room type: shared room
    "pg_Condominium",  # Property group: condo
    "pg_House",  # Property group: house
    "pg_Other",  # Property group: other
    "pg_Townhouse",  # Property group: townhouse
]

### Non-Spatial Regression, A (Very) Quick Refresh

Before we discuss how to explicitly include space into the linear regression framework, lwt us show how basic regression can be carried out in Python, and how on can begin to interperate the results. By no means is this a formal and complete introduction to regtrsion, so if that is what you are looking for see [GH06](https://doi.org/10.1017/cbo9780511790942) (in particular chapters 3 and 4, which provide a fantastic, non-spatial introduction).

The core idea of linear regression is to explain the variation in a given (*dependent*) variable as a linear function fo a collection of other (*explanatory*) variables. For example, in our case, we may want to express the price of a house as a finction of the number of bedroome it has and whether it is a condominium or not. At the individual level, we express this as:

$P_i = \alpha + \sum\limits_{k}\bf{X}_{ik}\beta_k + \epsilon_{i}$

where $P_i$ in the Airbnb price of house $i$ and $\bf{X}$ is a set of covariates that we use to explain such price (e.g., No. of bedrooms and condominium binary variable). $\beta$ is a vector of parameters that give us information about in which way (e.g., increases price, decreases) and to what extent (e.g., var_a is responsible for 20% of price), $\alpha$ is a constant term that explains the average price of a house with all other variable set to zero. The term $\epsilon_{i}$ is an error term and captures elemente that influence the price of a house not included in $\bf{X}$, We can also express this relation in matrix form, excluding sub-indicies for $i$, which yields:

$P = \alpha + \bf{X}\beta + \epsilon$

A regression can be seen as a multivariate extention fo bivariate cerrelations. Indeed one way to interpreate the $\beta_{k}$ coefficents in the equation above is as the degree of correlation between the explanatory variable $k$ and the dependent variable, *keeping all the other explanatory variables constant*. Whrn one calculates bivatiate correlations, the coefficent of a variable is picling up the forrelation between the variables, but it is also subsuming into it the variation associates eith other correlated variables - also called confounding factors. Regression allows us to isolate the distinct effect that a single variable can have on the dependent one, once we *control* for those other variables. 

Practically speaking, linear regressions in Python are rather streamlined and easy to work with. There are also several packages which will run them (e.g., statsmodels, scikit-learn, pysal). We will import the spreg module in Pysal:

In [4]:
from pysal.model import spreg

In the context of this notebook it makes sense to start with spreg, as that is the only library that will allow us to move into explicitly spatial econometric models. To fit the model specified in the equation above with $\bf{X}$ as the list defined, using ordinary least squares (OLS), we need only the following lien of code: 

In [6]:
# Fit the OLS model
m1 = spreg.OLS(
               db[["log_price"]].values,   # Dependent variable
               db[variable_names].values,  # Independent variables
               name_y="log_price",         # Dependent variable name
               name_x=variable_names,      # Independent variable names
              )

We use the command OLS, part of the spreg sub-package, and specify the dependent variable (the log of the price, so we can interpret results in terms of percentage change) and the explanatory iones. Note that both objects need to be arrays, we we extract them from the pandas dataframe object with .values.

In order to inspect the results of the model, we print the summary attribute:

In [7]:
print(m1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :   log_price                Number of Observations:        6110
Mean dependent var  :      4.9958                Number of Variables   :          11
S.D. dependent var  :      0.8072                Degrees of Freedom    :        6099
R-squared           :      0.6683
Adjusted R-squared  :      0.6678
Sum squared residual:     1320.15                F-statistic           :   1229.0564
Sigma-square        :       0.216                Prob(F-statistic)     :           0
S.E. of regression  :       0.465                Log likelihood        :   -3988.895
Sigma-square ML     :       0.216                Akaike info criterion :    7999.790
S.E of regression ML:      0.4648                Schwarz criterion     :    8073.685

------------------------------------------------------------

A full detailed explanation of the output is beyond the scope of this notebook, so we ficus on the televant bits for our main purpose. The focus will be on the Coefficente section, which gives us the estimates for $\beta_{k}$ in our model. In other words, these numbners express the relationship between each explanatory variable and the dependent one, once the effict of confounding variables has been accounted for. Keep in mind that regression is no magic; we are only dicounting the effect of confounding factors that we include in the model, not of *all* potintially confounding factors.

Results are largely ase expected: houses tend to be significantaly more expensive if they accomodate morte people (variable accomodates), if they have more bathrooms and bedrooms, and if they are a condominium or part of the "other" category of house type. Conversly, given a number of rooms, houses with more beds (i.e., listings that are more "crowded") tend to go for cheaper, as it is the case for properties where one does not rent the entire house but only a room (variable rt_Private_room) or even shares it (variable rt_Shared_room). Of course, you might conceptually doubt the assumption that it is possible to *arbitrarily* change the number of beds within an Airbnb without eventually changing the number of people it accomodates, but methods to address these concerns using *interaction effects* won't be discussed here. 